In [1]:
import numpy as np
import os
import sys
import pickle
from scipy.interpolate import interp1d

muscle_names = ['b1','b2','b3','i1','i2','iii1','iii24','iii3','hg1','hg2','hg3','hg4','freq']

c_muscle = [(0.5451,0.0,0.0),(1.0,0.0,0.0),(1.0,0.2706,0.0),
    (0.1176,0.5647,1.0),(0.0,0.0,1.0),(0.0,0.3922,0.0),
    (0.0,1.0,0.0),(0.0,1.0,0.4980),(0.2941,0.0,0.5098),
    (1.0,0.0,1.0),(0.7294,0.3333,0.8275),(1.0,0.0784,0.5765),(0.5,0.5,0.5)]

N_window = 9

dt_muscle = 0.015

filename = 'all_fly_m_data.p'

infile = open(filename,'rb')
data_dict = pickle.load(infile)
infile.close()

print(data_dict)

N_flies = len(data_dict)

t_data = []
kinefly_data = []
muscle_data_L = []
muscle_data_R = []

# Load data into lists:
for i in range(N_flies):
    
    kine_lr_i  = data_dict[i]['kine-lr']
    kine_l_i   = data_dict[i]['kine_l']
    kine_r_i   = data_dict[i]['kine_r']
    muscle_l_i = data_dict[i]['muscles_l']
    muscle_r_i = data_dict[i]['muscles_r']
    
    N_kine_i = kine_lr_i.shape[0]
    
    N_muscle_i = muscle_l_i[0].shape[0]
    
    t_muscle = np.linspace(0,dt_muscle*N_muscle_i,num=N_muscle_i)
    
    t_data.append(t_muscle)
    
    muscle_data_L_i = np.zeros((13,N_muscle_i))
    muscle_data_R_i = np.zeros((13,N_muscle_i))
    
    # Interpolate kinefly data:
    t_kine_i = np.linspace(0,dt_muscle*N_muscle_i,num=N_kine_i)
    
    kine_interp_lr = interp1d(t_kine_i,kine_lr_i,kind='cubic')
    kine_interp_l  = interp1d(t_kine_i,kine_l_i,kind='cubic')
    kine_interp_r  = interp1d(t_kine_i,kine_r_i,kind='cubic')
    
    kinefly_data_i = np.zeros((3,N_muscle_i))
    kinefly_data_i[0,:] = kine_interp_lr(t_muscle)
    kinefly_data_i[1,:] = kine_interp_l(t_muscle)
    kinefly_data_i[2,:] = kine_interp_r(t_muscle)
    
    kinefly_data.append(kinefly_data_i)
        
    # Load and scale muscle data:
    
    flight_stop_ind = (muscle_l_i[2]<0.0)|(muscle_l_i[1]<0.0)|(muscle_r_i[2]<0.0)|(muscle_r_i[1]<0.0)
    
    # Left muscles:
    b1_L   = muscle_l_i[0] #-muscle_l_i[3]
    b2_L   = muscle_l_i[1] #-muscle_l_i[3]
    b3_L   = muscle_l_i[2] #-muscle_l_i[3]
    i1_L   = muscle_l_i[8] #-muscle_l_i[3]
    i2_L   = muscle_l_i[9] #-muscle_l_i[3]
    iii1_L = muscle_l_i[10] #-muscle_l_i[3]
    iii2_L = muscle_l_i[11] #-muscle_l_i[3]
    iii3_L = muscle_l_i[12] #-muscle_l_i[3]
    hg1_L  = muscle_l_i[4] #-muscle_l_i[3]
    hg2_L  = muscle_l_i[5] #-muscle_l_i[3]
    hg3_L  = muscle_l_i[6] #-muscle_l_i[3]
    hg4_L  = muscle_l_i[7] #-muscle_l_i[3]
    
    b1L_sort = np.sort(b1_L)
    n_b1L = int(N_muscle_i*0.05)
    b1L_min = b1L_sort[n_b1L]
    b1L_max = b1L_sort[-n_b1L]
    muscle_data_L_i[0,:]  = (b1_L-b1L_min)/(b1L_max-b1L_min)
    
    b2L_sort = np.sort(b2_L)
    n_b2L = int(N_muscle_i*0.5)
    m_b2L = int(N_muscle_i*0.001)
    b2L_min = b2L_sort[n_b2L]
    b2L_max = b2L_sort[-m_b2L]
    muscle_data_L_i[1,:]  = (b2_L-b2L_min)/(b2L_max-b2L_min)
    
    b3L_sort = np.sort(b3_L)
    n_b3L = int(N_muscle_i*0.05)
    b3L_min = b3L_sort[n_b3L]
    b3L_max = b3L_sort[-n_b3L]
    muscle_data_L_i[2,:]  = (b3_L-b3L_min)/(b3L_max-b3L_min)
    
    i1L_sort = np.sort(i1_L)
    n_i1L = int(N_muscle_i*0.05)
    i1L_min = i1L_sort[n_i1L]
    i1L_max = i1L_sort[-n_i1L]
    muscle_data_L_i[3,:]  = (i1_L-i1L_min)/(i1L_max-i1L_min)
    
    i2L_sort = np.sort(i2_L)
    n_i2L = int(N_muscle_i*0.05)
    i2L_min = i2L_sort[n_i2L]
    i2L_max = i2L_sort[-n_i2L]
    muscle_data_L_i[4,:]  = (i2_L-i2L_min)/(i2L_max-i2L_min)
    
    iii1L_sort = np.sort(iii1_L)
    n_iii1L = int(N_muscle_i*0.5)
    m_iii1L = int(N_muscle_i*0.01)
    iii1L_min = iii1L_sort[n_iii1L]
    iii1L_max = iii1L_sort[-m_iii1L]
    muscle_data_L_i[5,:]  = (iii1_L-iii1L_min)/(iii1L_max-iii1L_min)
    
    iii2L_sort = np.sort(iii2_L)
    n_iii2L = int(N_muscle_i*0.05)
    iii2L_min = iii2L_sort[n_iii2L]
    iii2L_max = iii2L_sort[-n_iii2L]
    muscle_data_L_i[6,:]  = (iii2_L-iii2L_min)/(iii2L_max-iii2L_min)
    
    iii3L_sort = np.sort(iii3_L)
    n_iii3L = int(N_muscle_i*0.05)
    iii3L_min = iii3L_sort[n_iii3L]
    iii3L_max = iii3L_sort[-n_iii3L]
    muscle_data_L_i[7,:]  = (iii3_L-iii3L_min)/(iii3L_max-iii3L_min)
    
    hg1L_sort = np.sort(hg1_L)
    n_hg1L = int(N_muscle_i*0.05)
    hg1L_min = hg1L_sort[n_hg1L]
    hg1L_max = hg1L_sort[-n_hg1L]
    muscle_data_L_i[8,:]  = (hg1_L-hg1L_min)/(hg1L_max-hg1L_min)
    
    hg2L_sort = np.sort(hg2_L)
    n_hg2L = int(N_muscle_i*0.05)
    hg2L_min = hg2L_sort[n_hg2L]
    hg2L_max = hg2L_sort[-n_hg2L]
    muscle_data_L_i[9,:]  = (hg2_L-hg2L_min)/(hg2L_max-hg2L_min)
    
    hg3L_sort = np.sort(hg3_L)
    n_hg3L = int(N_muscle_i*0.05)
    hg3L_min = hg3L_sort[n_hg3L]
    hg3L_max = hg3L_sort[-n_hg3L]
    muscle_data_L_i[10,:]  = (hg3_L-hg3L_min)/(hg3L_max-hg3L_min)
    
    hg4L_sort = np.sort(hg4_L)
    n_hg4L = int(N_muscle_i*0.05)
    hg4L_min = hg4L_sort[n_hg4L]
    hg4L_max = hg4L_sort[-n_hg4L]
    muscle_data_L_i[11,:]  = (hg4_L-hg4L_min)/(hg4L_max-hg4L_min)
    
    muscle_data_L_i[12,:] = 0.5*np.ones(N_muscle_i)
    
    # Right muscles:
    b1_R   = muscle_r_i[0] #-muscle_r_i[3]
    b2_R   = muscle_r_i[1] #-muscle_r_i[3]
    b3_R   = muscle_r_i[2] #-muscle_r_i[3]
    i1_R   = muscle_r_i[8] #-muscle_r_i[3]
    i2_R   = muscle_r_i[9] #-muscle_r_i[3]
    iii1_R = muscle_r_i[10] #-muscle_r_i[3]
    iii2_R = muscle_r_i[11] #-muscle_r_i[3]
    iii3_R = muscle_r_i[12] #-muscle_r_i[3]
    hg1_R  = muscle_r_i[4] #-muscle_r_i[3]
    hg2_R  = muscle_r_i[5] #-muscle_r_i[3]
    hg3_R  = muscle_r_i[6] #-muscle_r_i[3]
    hg4_R  = muscle_r_i[7] #-muscle_r_i[3]
    
    b1R_sort = np.sort(b1_R)
    n_b1R = int(N_muscle_i*0.05)
    b1R_min = b1R_sort[n_b1R]
    b1R_max = b1R_sort[-n_b1R]
    muscle_data_R_i[0,:]  = (b1_R-b1R_min)/(b1R_max-b1R_min)
    
    b2R_sort = np.sort(b2_R)
    n_b2R = int(N_muscle_i*0.5)
    m_b2R = int(N_muscle_i*0.001)
    b2R_min = b2R_sort[n_b2R]
    b2R_max = b2R_sort[-m_b2R]
    muscle_data_R_i[1,:]  = (b2_R-b2R_min)/(b2R_max-b2R_min)
    
    b3R_sort = np.sort(b3_R)
    n_b3R = int(N_muscle_i*0.05)
    b3R_min = b3R_sort[n_b3R]
    b3R_max = b3R_sort[-n_b3R]
    muscle_data_R_i[2,:]  = (b3_R-b3R_min)/(b3R_max-b3R_min)
    
    i1R_sort = np.sort(i1_R)
    n_i1R = int(N_muscle_i*0.05)
    i1R_min = i1R_sort[n_i1R]
    i1R_max = i1R_sort[-n_i1R]
    muscle_data_R_i[3,:]  = (i1_R-i1R_min)/(i1R_max-i1R_min)
    
    i2R_sort = np.sort(i2_R)
    n_i2R = int(N_muscle_i*0.05)
    i2R_min = i2R_sort[n_i2R]
    i2R_max = i2R_sort[-n_i2R]
    muscle_data_R_i[4,:]  = (i2_R-i2R_min)/(i2R_max-i2R_min)
    
    iii1R_sort = np.sort(iii1_R)
    n_iii1R = int(N_muscle_i*0.5)
    m_iii1R = int(N_muscle_i*0.001)
    iii1R_min = iii1R_sort[n_iii1R]
    iii1R_max = iii1R_sort[-m_iii1R]
    muscle_data_R_i[5,:]  = (iii1_R-iii1R_min)/(iii1R_max-iii1R_min)
    
    iii2R_sort = np.sort(iii2_R)
    n_iii2R = int(N_muscle_i*0.05)
    iii2R_min = iii2R_sort[n_iii2R]
    iii2R_max = iii2R_sort[-n_iii2R]
    muscle_data_R_i[6,:]  = (iii2_R-iii2R_min)/(iii2R_max-iii2R_min)
    
    iii3R_sort = np.sort(iii3_R)
    n_iii3R = int(N_muscle_i*0.05)
    iii3R_min = iii3R_sort[n_iii3R]
    iii3R_max = iii3R_sort[-n_iii3R]
    muscle_data_R_i[7,:]  = (iii3_R-iii3R_min)/(iii3R_max-iii3R_min)
    
    hg1R_sort = np.sort(hg1_R)
    n_hg1R = int(N_muscle_i*0.05)
    hg1R_min = hg1R_sort[n_hg1R]
    hg1R_max = hg1R_sort[-n_hg1R]
    muscle_data_R_i[8,:]  = (hg1_R-hg1R_min)/(hg1R_max-hg1R_min)
    
    hg2R_sort = np.sort(hg2_R)
    n_hg2R = int(N_muscle_i*0.05)
    hg2R_min = hg2R_sort[n_hg2R]
    hg2R_max = hg2R_sort[-n_hg2R]
    muscle_data_R_i[9,:]  = (hg2_R-hg2R_min)/(hg2R_max-hg2R_min)
    
    hg3R_sort = np.sort(hg3_R)
    n_hg3R = int(N_muscle_i*0.05)
    hg3R_min = hg3R_sort[n_hg3R]
    hg3R_max = hg3R_sort[-n_hg3R]
    muscle_data_R_i[10,:]  = (hg3_R-hg3R_min)/(hg3R_max-hg3R_min)
    
    hg4R_sort = np.sort(hg4_R)
    n_hg4R = int(N_muscle_i*0.05)
    hg4R_min = hg4R_sort[n_hg4R]
    hg4R_max = hg4R_sort[-n_hg4R]
    muscle_data_R_i[11,:]  = (hg4_R-hg4R_min)/(hg4R_max-hg4R_min)
    
    muscle_data_R_i[12,:] = 0.5*np.ones(N_muscle_i)
    
    muscle_data_L.append(muscle_data_L_i)
    muscle_data_R.append(muscle_data_R_i)

[{'fly_n': 0, 'fly_datapaths': '/Users/fponce/Documents/muscleim_arena_opto/MI_063022/MI_2022-07-06-19-01-30.hdf5', 'muscle_key': ['b1', 'b2', 'b3', 'bkg', 'hg1', 'hg2', 'hg3', 'hg4', 'i1', 'i2', 'iii1', 'iii24', 'iii3', 'nm', 'pr', 'tpd'], 'kine-lr': array([12.78407187, 12.92765486, 13.07937329, ...,  0.80730387,
        1.96223638,  3.18938412]), 'kine_l': array([62.55081681, 63.07857808, 63.63375659, ..., 67.51734942,
       67.77819147, 68.04945181]), 'muscles_l': [array([0.7875573 , 0.7870736 , 0.77787115, ..., 0.71996058, 0.75024417,
       0.74076498]), array([1.7113601 , 1.69185734, 1.70306857, ..., 1.65322901, 1.70462548,
       1.67737702]), array([1.71176291, 1.6655031 , 1.67094274, ..., 1.50699644, 1.44289262,
       1.425809  ]), array([12.75640297, 12.70242045, 12.16047165, ..., 13.72806512,
       13.06958554, 13.51717756]), array([3.59467363, 3.60293338, 3.67891217, ..., 5.23929983, 5.06946176,
       4.7082048 ]), array([0.82490897, 0.56881256, 0.92439649, ..., 0.50088

In [2]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec

linewidth = 0.8
plot_loc = 'C:/Users/Johan/Documents/data_Francesca/plots'

for i in range(N_flies):
    
    fig = plt.figure()
    fig.suptitle('fly: '+str(i+1))
    fig.set_size_inches(24,18)
    
    gs = GridSpec(11,1)
    
    ax_lr   = fig.add_subplot(gs[0,0])
    ax_l    = fig.add_subplot(gs[1])
    ax_r    = fig.add_subplot(gs[6])
    ax_bL   = fig.add_subplot(gs[2])
    ax_iL   = fig.add_subplot(gs[3])
    ax_iiiL = fig.add_subplot(gs[4])
    ax_hgL  = fig.add_subplot(gs[5])
    ax_bR   = fig.add_subplot(gs[7])
    ax_iR   = fig.add_subplot(gs[8])
    ax_iiiR = fig.add_subplot(gs[9])
    ax_hgR  = fig.add_subplot(gs[10])
    
    ax_lr.plot(t_data[i],kinefly_data[i][0,:],color='k',linewidth=linewidth)
    ax_l.plot(t_data[i],kinefly_data[i][1,:],color='r',linewidth=linewidth)
    
    ax_bL.plot(t_data[i],muscle_data_L[i][0,:],color=c_muscle[0],linewidth=linewidth,label=muscle_names[0])
    ax_bL.plot(t_data[i],muscle_data_L[i][1,:],color=c_muscle[1],linewidth=linewidth,label=muscle_names[1])
    ax_bL.plot(t_data[i],muscle_data_L[i][2,:],color=c_muscle[2],linewidth=linewidth,label=muscle_names[2])
    ax_bL.legend(loc='right')
    ax_iL.plot(t_data[i],muscle_data_L[i][3,:],color=c_muscle[3],linewidth=linewidth,label=muscle_names[3])
    ax_iL.plot(t_data[i],muscle_data_L[i][4,:],color=c_muscle[4],linewidth=linewidth,label=muscle_names[4])
    ax_iL.legend(loc='right')
    ax_iiiL.plot(t_data[i],muscle_data_L[i][5,:],color=c_muscle[5],linewidth=linewidth,label=muscle_names[5])
    ax_iiiL.plot(t_data[i],muscle_data_L[i][6,:],color=c_muscle[6],linewidth=linewidth,label=muscle_names[6])
    ax_iiiL.plot(t_data[i],muscle_data_L[i][7,:],color=c_muscle[7],linewidth=linewidth,label=muscle_names[7])
    ax_iiiL.legend(loc='right')
    ax_hgL.plot(t_data[i],muscle_data_L[i][8,:],color=c_muscle[8],linewidth=linewidth,label=muscle_names[8])
    ax_hgL.plot(t_data[i],muscle_data_L[i][9,:],color=c_muscle[9],linewidth=linewidth,label=muscle_names[9])
    ax_hgL.plot(t_data[i],muscle_data_L[i][10,:],color=c_muscle[10],linewidth=linewidth,label=muscle_names[10])
    ax_hgL.plot(t_data[i],muscle_data_L[i][11,:],color=c_muscle[11],linewidth=linewidth,label=muscle_names[11])
    ax_hgL.legend(loc='right')
    
    ax_r.plot(t_data[i],kinefly_data[i][2,:],color='b',linewidth=linewidth)
    
    ax_bR.plot(t_data[i],muscle_data_R[i][0,:],color=c_muscle[0],linewidth=linewidth,label=muscle_names[0])
    ax_bR.plot(t_data[i],muscle_data_R[i][1,:],color=c_muscle[1],linewidth=linewidth,label=muscle_names[1])
    ax_bR.plot(t_data[i],muscle_data_R[i][2,:],color=c_muscle[2],linewidth=linewidth,label=muscle_names[2])
    ax_bR.legend(loc='right')
    ax_iR.plot(t_data[i],muscle_data_R[i][3,:],color=c_muscle[3],linewidth=linewidth,label=muscle_names[3])
    ax_iR.plot(t_data[i],muscle_data_R[i][4,:],color=c_muscle[4],linewidth=linewidth,label=muscle_names[4])
    ax_iR.legend(loc='right')
    ax_iiiR.plot(t_data[i],muscle_data_R[i][5,:],color=c_muscle[5],linewidth=linewidth,label=muscle_names[5])
    ax_iiiR.plot(t_data[i],muscle_data_R[i][6,:],color=c_muscle[6],linewidth=linewidth,label=muscle_names[6])
    ax_iiiR.plot(t_data[i],muscle_data_R[i][7,:],color=c_muscle[7],linewidth=linewidth,label=muscle_names[7])
    ax_iiiR.legend(loc='right')
    ax_hgR.plot(t_data[i],muscle_data_R[i][8,:],color=c_muscle[8],linewidth=linewidth,label=muscle_names[8])
    ax_hgR.plot(t_data[i],muscle_data_R[i][9,:],color=c_muscle[9],linewidth=linewidth,label=muscle_names[9])
    ax_hgR.plot(t_data[i],muscle_data_R[i][10,:],color=c_muscle[10],linewidth=linewidth,label=muscle_names[10])
    ax_hgR.plot(t_data[i],muscle_data_R[i][11,:],color=c_muscle[11],linewidth=linewidth,label=muscle_names[11])
    ax_hgR.legend(loc='right')
    
    ax_lr.set_ylim([-30,30])
    ax_l.set_ylim([0,90])
    ax_r.set_ylim([0,90])
    ax_bL.set_ylim([-0.5,1.5])
    ax_iL.set_ylim([-0.5,1.5])
    ax_iiiL.set_ylim([-0.5,1.5])
    ax_hgL.set_ylim([-0.5,1.5])
    ax_bR.set_ylim([-0.5,1.5])
    ax_iR.set_ylim([-0.5,1.5])
    ax_iiiR.set_ylim([-0.5,1.5])
    ax_hgR.set_ylim([-0.5,1.5])
    
    file_name = plot_loc + '/data_fly_'+str(i+1)+'.svg'
    fig.savefig(file_name, dpi=200)
    plt.close(fig)
    